In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd
import dask.dataframe as dd
from transformers import AutoTokenizer
import pyarrow as pa

In [4]:
X_train = pd.read_csv('drive/MyDrive/BERT Sentiment/CSVs/X_train_full.csv')
#X_test = pd.read_csv('drive/MyDrive/BERT Sentiment/CSVs/X_test_full.csv')

In [5]:
X_train.columns = ['index','review']
#X_test.columns = ['index','review']

In [6]:
X_train.set_index(keys='index',inplace=True)
#X_test.set_index('index',inplace=True)
X_train.head()

,review
index,
2522958,Doggy Hoots Cracklers: Not as enthusiastic as ...
1160125,great product: Perfect product for freezing mi...
861121,"Awesome Trimmer, Lousy Power: It pains me to g..."
300957,"Oh, T.I. You should be ASHAMED to even call t..."
1610389,This is one of my favorite movies: Forest Gump...


In [7]:
# Load HuggingFace Tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [11]:
# Tokenize reviews
test_inputs = tokenizer(list(X_test['review']),truncation=True,padding=True)

In [12]:
test_inputs.keys()


dict_keys(['input_ids', 'attention_mask'])

In [13]:
# Examining reviews with unknown tokens
has_unknown = []
for i, input in enumerate(test_inputs['input_ids']):
  for tok in input:
    # Searches for an [UNK] token
    if tok == 100:
      # Prints entire line with padding characters removed
      print(f"input {i}: {tokenizer.decode(input).replace(' [PAD]','')}.")
      has_unknown.append(i)
      break

input 34945: [CLS] fat? copy? i think not... : do these look fat to you? no!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 1do these look like bratz? no!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! she shows some skin? yes!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! is that horribly wrong? no!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ( do you show skin? yes!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! is this doll perfect? [UNK]!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! [SEP].
input 138420: [CLS] [UNK] [SEP].
input 263693: [CLS] 3. 75 star review ( i'm an engineer..... what can i say, i like precision! ) : - ) \ n \ nback for another run at vapiano's, and another solid effort. the trick is to go early before the banking crowd overruns the place..... so 1130 is ideal. \ n \ ni had the americana pizza, with marinara, fresh mozzerella, pepperoni and italian sausage, and i had them add onions and mushrooms. neopolitan style, not \ " ny \ " style ( with that cru

In [14]:
# Checking the originals of those reviews
for index in has_unknown:
  print(X_test.iloc[index]['review'])

FAT? COPY? I THINK NOT...: Do these look fat to you?NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1DO THESE LOOK LIKE BRATZ?NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!SHE SHOWS SOME SKIN?YES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!IS THAT HORRIBLY WRONG?NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!(do you show skin?YES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!IS THIS DOLL PERFECT?YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm1
3.75 Star Review (I'm an Engineer.....what can I say, I like precision!)  :-)\n\nBack for another run at Vapiano's, and another solid effort. The trick is to go early before the Banking crowd overruns the place.....so 1130 i

In [ ]:
# Sample from above parenthetical)
untokenizable_sample = 'mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm'
len(untokenizable_sample)




157

It looks like the primary source of unknown tokens are excessively repeated characters, such as in "mmmmmmmmmm..." with 157 m's.

In [ ]:
type(test_inputs)

transformers.tokenization_utils_base.BatchEncoding

In [16]:
X_test.columns

Index(['review'], dtype='object')

In [17]:
# Create a dask dataframe of the tokenized reviews
test_input_ddf = dd.from_dict({'index': X_test.index,
                               'input_ids': test_inputs['input_ids'],
                               'attention_mask': test_inputs['attention_mask']},
                              npartitions=10)

In [18]:
# Check that inputs match up
test_input_ddf.head(5)


,index,input_ids,attention_mask
0,886662,"[101, 2307, 7692, 1024, 2023, 2003, 1037, 2307...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,112830,"[101, 2054, 2019, 12476, 24385, 999, 1032, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2320316,"[101, 2204, 2096, 2009, 6354, 1006, 2005, 2026...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,566879,"[101, 1045, 2293, 2023, 2173, 1012, 1045, 2134...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1666132,"[101, 2987, 1005, 1056, 2377, 2006, 1037, 4966...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [22]:
# Check that the first shuffled index decodes as expected
tokenizer.decode(test_input_ddf.loc[test_input_ddf['index']==886662,'input_ids'].compute()[0])

'[CLS] great resource : this is a great down to earth discussion of late talking children. there have been many criticisms of this book regarding some of the treatment suggestions. think of the book as a transcript of a " parents of late talking children " support group meeting and you won\'t go too far wrong. the author is a very intelligent individual who is writing this as a labor of love. for us, the most important thing was just to be able to read story after story of people who have kids like ours. the treatment suggestions are great tools to spark your thinking but use your brain and if you are unsure of something talk to your doctor or speech therapist. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [23]:
X_test.head(1)

,review
index,
886662,Great resource: This is a great down to earth ...


In [8]:
schema = pa.schema([('index', pa.int64()),
                    ('input_ids', pa.list_(pa.int64())),
                    ('attention_mask', pa.list_(pa.int64()))])

In [24]:
test_input_ddf.to_parquet('drive/MyDrive/BERT Sentiment/CSVs/test_inputs.parquet',schema=schema)


In [8]:
# X_train is too big to hold in memory. Going to need to batch them.
batch_size = len(X_train)//15 + 1
batch_size

278857

In [10]:
# Tokenize reviews in batches
for i in range(15):
  start = i*batch_size
  end = min((i+1)*batch_size,len(X_train))
  train_inputs = tokenizer(list(X_train['review'].iloc[start:end]),truncation=True,padding=True)
  train_indices = X_train.index[start:end]
  train_input_ddf = dd.from_dict({'index': train_indices,
                                  'input_ids': train_inputs['input_ids'],
                                  'attention_mask': train_inputs['attention_mask']},
                                 npartitions=10)
  train_input_ddf.to_parquet(f'drive/MyDrive/BERT Sentiment/CSVs/train_inputs_{i}.parquet',schema=schema)


In [11]:
train_input_ddf.head(5)

,index,input_ids,attention_mask
0,1736989,"[101, 3828, 2115, 2051, 1998, 2115, 2769, 1012...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,303632,"[101, 1996, 2034, 3573, 1997, 2049, 2785, 1999...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2216184,"[101, 6581, 3643, 1024, 12476, 4966, 2447, 101...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,2994343,"[101, 2204, 2801, 1012, 1012, 1012, 1024, 1996...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,3397437,"[101, 2178, 2012, 13267, 10514, 21442, 1024, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
# Test that the dataframes are correctly labeled


In [15]:
# Test that the dataframes are correctly labeled.
train_inputs = tokenizer(list(X_train['review'].iloc[batch_size:2*batch_size]),truncation=True,padding=True)
train_indices = X_train.index[batch_size:2*batch_size]
train_input_ddf = dd.from_dict({'index': train_indices,
                                  'input_ids': train_inputs['input_ids'],
                                  'attention_mask': train_inputs['attention_mask']},
                                 npartitions=10)

In [17]:
train_input_ddf.head()

,index,input_ids,attention_mask
0,2025630,"[101, 6331, 1024, 2293, 1996, 6331, 1998, 1996...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,198034,"[101, 2023, 2003, 2011, 2521, 2026, 5440, 2173...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2418716,"[101, 1996, 2190, 1997, 1996, 6386, 1005, 2220...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1371917,"[101, 1043, 2135, 17119, 3170, 2192, 7242, 102...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1524295,"[101, 5013, 7641, 2041, 1011, 1011, 3828, 2115...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [18]:
tokenizer.decode(train_input_ddf.loc[train_input_ddf['index']==2025630,'input_ids'].compute()[0])

"[CLS] bible : love the bible and the layout. just wish the cover option would have been better. this one is cheap. hardcover same. maybe if it had come in imitation leather for a little more in money i would have bought that one but that was not one of the options. if my boys get 2 years out of this bible i'm lucky. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [19]:
X_train.iloc[batch_size]

review    Bible: Love the Bible and the layout. Just wis...
Name: 2025630, dtype: object